In [14]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning
ERROR: file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import random

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train
from src.models.train_model import train_conf_pen
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

from src.models.reg import reg_loss

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [3]:
def make_preg_loss(L_cls, L_preg, mu, A_hat):
    """  """
    def l(data, pred):
        L_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        L_preg = L_preg(pred[data.train_mask], A_hat@pred[data.reg_mask])
        return  + mu * L_preg(x,y)
    return l


def make_confidence_penalty_loss(L_cls, beta):
    """  """
    def l(data, pred):
        L_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        H = -torch.sum(pred[data.reg_mask] * torch.log2(pred[data.reg_mask])) / pred.shape[0]

        return L_cls - beta * H
    return l

In [4]:
L_cls = lambda x, y: F.nll_loss(torch.log(x), y),
L_preg = lambda x, y: F.cross_entropy(x, y),


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
# no normalization, using n != 0 and m!= 0 instead
# doesnt work, just use normalization instead
# dataset = Planetoid(root='data/Planetoid', name='Cora')

data = dataset[0].to(device)
data = random_splits(data, 50, 20)
splits = data.train_mask, data.val_mask, data.test_mask

In [7]:
def set_reg_mask(data, A):
    reg_indeces = random.sample(range(data.x.shape[0]), k=A)
    reg_mask = torch.zeros(len(data.y), dtype=torch.bool)
    for i in reg_indeces:
        reg_mask[i] = True
    data.reg_mask = reg_mask
    return data

data = set_reg_mask(data, 20)
print(data.reg_mask.sum())

tensor(20)


In [13]:
from src.models.reg import compute_a_hat

def make_preg_loss(L_cls, L_preg, mu, A_hat):
    """  """
    def l(data, Z):
        P = F.softmax(Z[data.train_mask], dim=1)
        Q = F.softmax((A_hat@Z)[data.train_mask], dim=1)
        Y = data.y[data.train_mask]
        M = data.train_mask.sum()
        N = data.train_mask.shape[0]

        l_cls = L_cls(P, Y)
        l_preg = L_preg(P, Q)

        return - 1 / M * l_cls + mu / N * l_preg
    return l


def make_confidence_penalty_loss(L_cls, beta):
    """  """
    def l(data, pred):
        l_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        H = -torch.sum(pred[data.reg_mask] * torch.log2(pred[data.reg_mask])) / pred.shape[0]

        return l_cls - beta * H
    return l


a_hat = compute_a_hat(data)

L_cls = lambda x, y: F.nll_loss(torch.log(x), y)
L_preg = lambda x, y: F.cross_entropy(x, y)

l = make_preg_loss(L_cls, L_preg, 0.2, a_hat)


gcn_model = GCN0(num_node_features=dataset.num_node_features,
                num_classes=dataset.num_classes) \
                .to(device)

pred = gcn_model(data)
comped_l = l(data, pred)
comped_l_old = reg_loss(
            pred=F.softmax(pred, dim=1),
            prop=F.softmax(a_hat@pred, dim=1), # 
            target=data.y[data.train_mask], # we assume to have ground-truth labels on only a subset of the dataset
            train_mask=data.train_mask, # hence, we also provide the train_mask, to know what nodes have labels
            preg_mask=torch.ones_like(data.train_mask, dtype=torch.bool),
            L_cls=lambda x, y: F.nll_loss(torch.log(x), y),
            L_preg=lambda x, y: F.cross_entropy(x, y),
            mu=0.2,
            )
print(comped_l)
print(comped_l_old)



tensor(2.3356, grad_fn=<AddBackward0>)
tensor(2.3356, grad_fn=<AddBackward0>)


/tmp/ipykernel_10823/777732340.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  l_cls = L_cls(F.softmax(pred[data.train_mask]), data.y[data.train_mask])
/tmp/ipykernel_10823/777732340.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  l_preg = L_preg(F.softmax(pred[data.reg_mask]), F.softmax((A_hat@pred)[data.reg_mask]))
